In [1]:
from analogainas.search_spaces.config_space import ConfigSpace
## Default Search Space
# We will be using the default search space
CS = ConfigSpace()

In [2]:
# You can extract the full list of hyperparameters using:
CS.get_hyperparameters()

['out_channel0', 'M', 'R1', 'R2', 'R3', 'R4', 'R5', 'convblock1', 'widenfact1', 'B1', 'convblock2', 'widenfact2', 'B2', 'convblock3', 'widenfact3', 'B3', 'convblock4', 'widenfact4', 'B4', 'convblock5', 'widenfact5', 'B5']


In [3]:
# Compute the size of the search space
CS.compute_cs_size()

31850496

In [ ]:
# Load the evaluator
from analogainas.evaluators.xgboost import XGBoostEvaluator
evaluator = XGBoostEvaluator()
from analogainas.search_algorithms.ea_optimized import EAOptimizer
from analogainas.search_algorithms.worker import Worker
NB_RUN = 5

optimizer = EAOptimizer(evaluator, population_size=20, nb_iter=10)
worker = Worker(CS, optimizer=optimizer, runs=NB_RUN)
worker.search()

In [ ]:
worker.result_summary()
best_config = worker.best_config

In [4]:
best_config = {'out_channel0': 16,
 'M': 4,
 'R1': 2,
 'R2': 0,
 'R3': 1,
 'R4': 2,
 'R5': 0,
 'convblock1': 1,
 'widenfact1': 0.5330917238012098,
 'B1': 2,
 'convblock2': 2,
 'widenfact2': 0.5289583249829364,
 'B2': 1,
 'convblock3': 2,
 'widenfact3': 0.6765230043009361,
 'B3': 3,
 'convblock4': 2,
 'widenfact4': 0.7745942817441474,
 'B4': 1,
 'convblock5': 0,
 'widenfact5': 0,
 'B5': 0
 }

In [6]:
from analogainas.search_spaces.resnet_macro_architecture import Network
# from analogainas.search_spaces.train import train_config_unet
from analogainas.utils import *

import torch
# Building the Optimal Sub-network
model = Network(best_config)
# input_tensor = torch.rand((1, 3, 64, 64))
# output = model(input_tensor)
#train_config_unet()

In [7]:
from torchsummary import summary
summary(model, input_size=(3, 32, 32))

RuntimeError: output padding must be smaller than either stride or dilation, but got output_padding_height: 1 output_padding_width: 1 stride_height: 1 stride_width: 1 dilation_height: 1 dilation_width: 1

In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

In [ ]:
best_config_hc = {'out_channel0': 16,
 'M': 4,
 'R1': 2,
 'R2': 1,
 'R3': 2,
 'R4': 1,
 'R5': 0,
 'convblock1': 2,
 'widenfact1': 0.7939218808815371,
 'B1': 2,
 'convblock2': 2,
 'widenfact2': 0.5256597823796261,
 'B2': 4,
 'convblock3': 1,
 'widenfact3': 0.5364357109292749,
 'B3': 2,
 'convblock4': 2,
 'widenfact4': 0.5073309835858459,
 'B4': 3,
 'convblock5': 0,
 'widenfact5': 0,
 'B5': 0}

In [ ]:
print(model)

In [ ]:
# Metric Functions for Training Nuclei Segmentation

# IOU Score and DICE Coefficients
import numpy as np
import torch
import torch.nn.functional as F


def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)


def dice_coef(output, target):
    smooth = 1e-5

    output = torch.sigmoid(output).view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)

In [ ]:
# Utility functions

import argparse
def str2bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
# Loss Functions for Segmentation
import torch
import torch.nn as nn
import torch.nn.functional as F

class BCEDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        bce = F.binary_cross_entropy_with_logits(input, target)
        smooth = 1e-5
        input = torch.sigmoid(input)
        num = target.size(0)
        input = input.view(num, -1)
        target = target.view(num, -1)
        intersection = (input * target)
        dice = (2. * intersection.sum(1) + smooth) / (input.sum(1) + target.sum(1) + smooth)
        dice = 1 - dice.sum() / num
        return 0.5 * bce + dice


class LovaszHingeLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        input = input.squeeze(1)
        target = target.squeeze(1)
        loss = lovasz_hinge(input, target, per_image=True)

        return loss

In [ ]:
config = {
        'spatial_dims': 2,  # it's 2 for 2D images; would be 3 for 3D images
        'in_channels': 3,   # e.g., 3 for RGB input images
        'out_channels': 1,  # e.g., 1 for binary segmentation tasks
        'channels': (16, 32, 64, 128, 256), # Number of channels in the inner layers
        'strides': (2, 2, 2, 2),
        'name': None,
        'epochs': 200,
        'batch_size': 64,
        'arch': 'UNet',
        'deep_supervision': False,
        'input_channels': 3,
        'num_classes': 1,
        'input_w': 96,
        'input_h': 96,
        'loss': 'BCEDiceLoss',
        'dataset': 'dsb2018_96',
        'img_ext': '.png',
        'mask_ext': '.png',
        'optimizer': 'SGD',
        'lr': 1e-3,
        'momentum': 0.9,
        'weight_decay': 1e-4,
        'nesterov': False,
        'scheduler': 'CosineAnnealingLR',
        'min_lr': 1e-5,
        'factor': 0.1,
        'patience': 2,
        'milestones': '1,2',
        'gamma': 2/3,
        'early_stopping': -1,
        'num_workers': 4
    }

In [ ]:
# Digital Training imports
import argparse
import os
from collections import OrderedDict
from glob import glob

import pandas as pd
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import yaml
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from tqdm import tqdm


In [ ]:
LOSS_NAMES = ['BCEDiceLoss', 'LovaszHingeLoss']
LOSS_NAMES.append('BCEWithLogitsLoss')

def train(config, train_loader, model, criterion, optimizer):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    model.train()

    pbar = tqdm(total=len(train_loader))
    for input, target, _ in train_loader:
        input = input.cuda()
        target = target.cuda()

        # compute output
        if config['deep_supervision']:
            outputs = model(input)
            loss = 0
            for output in outputs:
                loss += criterion(output, target)
            loss /= len(outputs)
            iou = iou_score(outputs[-1], target)
        else:
            output = model(input)
            loss = criterion(output, target)
            iou = iou_score(output, target)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])

def validate(config, val_loader, model, criterion):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        pbar = tqdm(total=len(val_loader))
        for input, target, _ in val_loader:
            input = input.cuda()
            target = target.cuda()

            # compute output
            if config['deep_supervision']:
                outputs = model(input)
                loss = 0
                for output in outputs:
                    loss += criterion(output, target)
                loss /= len(outputs)
                iou = iou_score(outputs[-1], target)
            else:
                output = model(input)
                loss = criterion(output, target)
                iou = iou_score(output, target)

            avg_meters['loss'].update(loss.item(), input.size(0))
            avg_meters['iou'].update(iou, input.size(0))

            postfix = OrderedDict([
                ('loss', avg_meters['loss'].avg),
                ('iou', avg_meters['iou'].avg),
            ])
            pbar.set_postfix(postfix)
            pbar.update(1)
        pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])




In [ ]:
# define loss function (criterion)
if config['loss'] == 'BCEWithLogitsLoss':
    criterion = nn.BCEWithLogitsLoss().cuda()
elif config['loss'] == 'BCEDiceLoss':
    criterion = BCEDiceLoss().cuda()
else:
    criterion = LovaszHingeLoss().cuda()

cudnn.benchmark = True

model = {} # Use your NAS architecture here

params = filter(lambda p: p.requires_grad, model.parameters())

if config['optimizer'] == 'Adam':
    optimizer = optim.Adam(
        params, lr=config['lr'], weight_decay=config['weight_decay'])
elif config['optimizer'] == 'SGD':
    optimizer = optim.SGD(params, lr=config['lr'], momentum=config['momentum'],
                          nesterov=config['nesterov'], weight_decay=config['weight_decay'])
else:
    raise NotImplementedError

if config['scheduler'] == 'CosineAnnealingLR':
    scheduler = lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config['epochs'], eta_min=config['min_lr'])
elif config['scheduler'] == 'ReduceLROnPlateau':
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=config['factor'], patience=config['patience'],
                                                verbose=1, min_lr=config['min_lr'])
elif config['scheduler'] == 'MultiStepLR':
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[int(e) for e in config['milestones'].split(',')], gamma=config['gamma'])
elif config['scheduler'] == 'ConstantLR':
    scheduler = None
else:
    raise NotImplementedError

In [ ]:
from monai.utils import first, set_determinism
from monai.transforms import (
    Compose, 
    RandRotate90, 
    RandFlip, 
    OneOf, 
    RandAdjustContrast, 
    RandGaussianNoise,  # MONAI does not directly support RandomBrightness, using Gaussian noise as an alternative
    RandGaussianSharpen,  # MONAI does not have a direct match for RandomContrast, using Gaussian sharpen as an alternative
    RandShiftIntensity,  # Alternative for adjusting brightness
    Resize,
    NormalizeIntensity
)

# Data loading code
img_ids = glob(os.path.join('inputs', config['dataset'], 'images', '*' + config['img_ext']))
img_ids = [os.path.splitext(os.path.basename(p))[0] for p in img_ids]

train_img_ids, val_img_ids = train_test_split(img_ids, test_size=0.2, random_state=41)


train_transform = Compose([
    RandRotate90(prob=0.5, spatial_axes=(0, 1)),  # Randomly rotates the images by 90 degrees with a default probability of 0.5
    RandFlip(prob=0.5, spatial_axis=0),  # Randomly flips the image along a given axis
    OneOf([  # Applies one of the transforms
        RandShiftIntensity(offsets=0.1, prob=1.0),  # Randomly shifts intensity for brightness adjustment
        RandAdjustContrast(prob=1.0),  # Randomly changes contrast
        RandGaussianSharpen(prob=1.0),  # Randomly sharpens the image
    ]),
    Resize(spatial_size=(config['input_h'], config['input_w'])),  # Resize images to a specified size
    NormalizeIntensity(nonzero=True, channel_wise=True)  # Normalize pixel values with channel-wise option
])

val_transform = Compose([
    Resize(spatial_size=(config['input_h'], config['input_w'])),  # Resize images to a specified size
    NormalizeIntensity(nonzero=True, channel_wise=True)  # Normalize pixel values with channel-wise option
])

from analogainas.search_spaces.dataloaders.Nuclei_Dataset import Dataset as Dataset

train_dataset = Dataset(
    img_ids=train_img_ids,
    img_dir=os.path.join('inputs', config['dataset'], 'images'),
    mask_dir=os.path.join('inputs', config['dataset'], 'masks'),
    img_ext=config['img_ext'],
    mask_ext=config['mask_ext'],
    num_classes=config['num_classes'],
    transform=train_transform)

val_dataset = Dataset(
    img_ids=val_img_ids,
    img_dir=os.path.join('inputs', config['dataset'], 'images'),
    mask_dir=os.path.join('inputs', config['dataset'], 'masks'),
    img_ext=config['img_ext'],
    mask_ext=config['mask_ext'],
    num_classes=config['num_classes'],
    transform=val_transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=config['num_workers'],
    drop_last=True)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=config['num_workers'],
    drop_last=False)

In [ ]:
# Digital Training Loop

log = OrderedDict([
    ('epoch', []),
    ('lr', []),
    ('loss', []),
    ('iou', []),
    ('val_loss', []),
    ('val_iou', []),
])

best_iou = 0
trigger = 0
for epoch in range(config['epochs']):
    print('Epoch [%d/%d]' % (epoch, config['epochs']))

    # train for one epoch
    train_log = train(config, train_loader, model, criterion, optimizer)
    # evaluate on validation set
    val_log = validate(config, val_loader, model, criterion)

    if config['scheduler'] == 'CosineAnnealingLR':
        scheduler.step()
    elif config['scheduler'] == 'ReduceLROnPlateau':
        scheduler.step(val_log['loss'])

    print('loss %.4f - iou %.4f - val_loss %.4f - val_iou %.4f'
          % (train_log['loss'], train_log['iou'], val_log['loss'], val_log['iou']))

    log['epoch'].append(epoch)
    log['lr'].append(config['lr'])
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])

    pd.DataFrame(log).to_csv('models/%s/log.csv' %
                              config['name'], index=False)

    trigger += 1

    if val_log['iou'] > best_iou:
        torch.save(model.state_dict(), 'models/%s/model.pth' %
                    config['name'])
        best_iou = val_log['iou']
        print("=> saved best model")
        trigger = 0

    # early stopping
    if config['early_stopping'] >= 0 and trigger >= config['early_stopping']:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()

In [ ]:
from analogainas.search_spaces.resnet_macro_architecture import Network
from analogainas.search_spaces.train import train_config_unet
from analogainas.utils import *

# Building the Optimal Sub-network
model = Network(config)